In [1]:
import urllib
import json
import pandas as pd
import vertica_python

In [2]:
#Забираем файл и преобразовываем его в json.
url = 'https://www.cbr-xml-daily.ru/daily_json.js'
response = urllib.request.urlopen(url)
data = json.loads(response.read())

In [17]:
valute = ('USD', 'EUR', 'CNY', 'INR', 'JPY')
atribute = ('ID','NumCode', 'CharCode', 'Nominal', 'Name', 'Value', 'Previous')
delimiter = '\x1F'
line_terminator = '\x1E'

<class 'str'>
<class 'str'>


In [4]:
table_header = 'valute.valute'
rej_table = 'valute.rej_table_valute'

In [5]:
connection_info = {'host': 'localhost',
             'port': 5433,
             'user': 'dbadmin',
             'password': '',
             # default throw error on invalid UTF-8 results
             'unicode_error': 'strict',
             # SSL is disabled by default
             'ssl': False,
             # autocommit is off by default
             'autocommit': True,
             # using server-side prepared statements is disabled by default
             'use_prepared_statements': False,
             # connection timeout is not enabled by default
             # 5 seconds timeout for a socket operation (Establishing a TCP connection or read/write operation)
             'connection_timeout': 5}


In [6]:
#This code only for experiment ma'faka
#with open('valute.csv', 'w', newline='', encoding='utf8') as csvfile:
    #data_writer = csv.writer(csvfile, delimiter = '\x1F', lineterminator = '\x1E')
    #for i in valute:
        #for j in atribute:
            #a = str(data['Valute'][i][j])
            #data_writer.writerow([a]) 
            #print(type(a), a)

In [7]:
df = pd.DataFrame(data['Valute'])
df = df.T

In [27]:
df['Nominal'] = df['Nominal'].astype(int)
df['Value'] = df['Value'].astype(float)
df['Previous'] = df['Previous'].astype(float)

In [28]:
df.to_csv('valute.csv', index=False, sep = delimiter, line_terminator = line_terminator,
            header=True,
            encoding='utf-8',
            na_rep=' ')

In [32]:
with vertica_python.connect(**connection_info) as connection:
        copy_req = """COPY {table_header}
                FROM STDIN PARSER
                fcsvparser(
                    type='traditional',
                    delimiter='{delimiter}',
                    header='true',
                    record_terminator='{line_terminator}',
                    reject_on_materialized_type_error = 'true'
                ) REJECTED DATA AS TABLE {rej_table};
            """.format(
            table_header=table_header,
            delimiter=delimiter,
            line_terminator=line_terminator,
            rej_table=rej_table
        )
        cur = connection.cursor()
        cur.execute("START TRANSACTION;")
        cur.execute("SET SESSION AUTOCOMMIT TO OFF;")
        with open('C:\\Users\\adyan.beltikov\\Desktop\\programs\\Glob_currency\\valute.csv', "rb") as fs:
            try:
                cur.copy(copy_req, fs, buffer_size=65536)
                cur.execute('select get_num_rejected_rows();')
                after = cur.fetchone()
                print(after)
                cur.execute("COMMIT;")
            except Exception as ex:
                print(ex)

[34]
